In [1]:
import pandas as pd
df=pd.read_csv('tiki.csv')
df.head()

,ID,Name Product,Price (VND),Review,Discount ( %)
0,0,Máy Hút Bụi Mini Cầm Tay DEERMA DX128C,699000,166,58.0
1,1,Máy Lọc Không Khí Sharp FP-J30E-B - Hàng chí...,1799000,70,54.0
2,2,"Combo 3 lõi lọc nước Karofi 1,2,3 chính hãng",120000,586,57.0
3,3,Bàn Ủi Khô Nagakawa NAG1502 - Màu Ghi - Hàng C...,200000,1,13.0
4,4,Máy Hút Bụi Cầm Tay Không Dây Damas XC628 - Ch...,379000,318,58.0
